In [88]:
import http.client, urllib.parse, json, string, pickle
from time import sleep

import pandas as pd
import numpy as np

from tqdm import tqdm

np.random.seed(42)

In [2]:
def read_data(datapath):
    n_errors = 0
    was_error = False

    with open(datapath) as f:
        xy_list = list()
        tokens = list()
        tags = list()
        
        for line in f:
            # format error handling:
            if was_error:
                if line != '\n':
                    continue
                else:
                    was_error = False

            items = line.strip('\n').strip('\t').split('\t')
            if len(items) == 2:
                token, tag = items
                if token[0].isdigit():
                    tokens.append('#')
                else:
                    # is you want static noise, add it here
                    tokens.append(token.lower())
                tags.append(tag)
            elif line == '\n':
                xy_list.append((tokens, tags,))
                tokens = list()
                tags = list()
            else:
                # format error handling
                print(line)
                n_errors += 1
                was_error = True
                continue
    if n_errors > 0:
        print('Reading is done with {} errors'.format(n_errors))
    return xy_list


In [56]:
def argmax_suggestion(suggestions):
    argmax = 0
    for i, s in enumerate(suggestions):
#         print(s)
        if s['score'] > suggestions[argmax]['score']:
            argmax = i
    return argmax

def spellsafe(text, spellchecker_output):
    assert '_type' in spellchecker_output, spellchecker_output
    assert spellchecker_output['_type'] == 'SpellCheck', spellchecker_output

    result = text
    changes = {}
    for token_info in spellchecker_output['flaggedTokens']:
        
        argmax = argmax_suggestion(token_info['suggestions'])
        suggestion = token_info['suggestions'][argmax]['suggestion']
        if ' ' in suggestion or ' ' in token_info['token']:
#         if len(suggestion.split(' ')) > 1:
            print('token: ', token_info['token'], '\tsuggest: ', suggestion)
            continue
        token = token_info['token'].strip(' ').lstrip(' ')
        changes[token] = suggestion
    
    for wrong, fixed in changes.items():
        result = result.replace(wrong, fixed)
    return result

def untokenize(tokens):
    text = ''
    was_apostrophe = False
    for token in tokens:
        if token == "'":
            was_apostrophe = True
            text += token
        elif token in string.punctuation:
            was_apostrophe = False
            text += token
        elif was_apostrophe:
            was_apostrophe = False
            text += token
        else:
            was_apostrophe = False
            text += ' ' + token
    text = text.lstrip(' ')
    return text

In [69]:
text = 'Solut à tout!'

data = {'text': text}

# NOTE: Replace this example key with a valid subscription key.
key = '7559af8fffdd432da93328d18b20e85a'

host = 'api.cognitive.microsoft.com'
path = '/bing/v7.0/spellcheck?'
params = 'mkt=fr-fr&mode=spell'

headers = {'Ocp-Apim-Subscription-Key': key,
'Content-Type': 'application/x-www-form-urlencoded'}

# The headers in the following example 
# are optional but should be considered as required:
#
# X-MSEdge-ClientIP: 999.999.999.999  
# X-Search-Location: lat: +90.0000000000000;long: 00.0000000000000;re:100.000000000000
# X-MSEdge-ClientID: <Client ID from Previous Response Goes Here>

conn = http.client.HTTPSConnection(host)
body = urllib.parse.urlencode(data)
conn.request ("POST", path + params, body, headers)
response = conn.getresponse()
output = json.loads(response.read())
print(output)

{'_type': 'SpellCheck', 'flaggedTokens': [{'offset': 0, 'token': 'Solut', 'type': 'UnknownToken', 'suggestions': [{'suggestion': 'salut', 'score': 1}]}], 'correctionType': 'High'}


In [5]:
data = {'text': 'Je sui contens'}
body = urllib.parse.urlencode(data)
conn.request ("POST", path + params, body, headers)
response = conn.getresponse()
output = json.dumps(json.loads(response.read()), indent=4)
print(output)

{
    "_type": "SpellCheck",
    "flaggedTokens": [
        {
            "offset": 3,
            "token": "sui",
            "type": "UnknownToken",
            "suggestions": [
                {
                    "suggestion": "suis",
                    "score": 1
                }
            ]
        },
        {
            "offset": 7,
            "token": "contens",
            "type": "UnknownToken",
            "suggestions": [
                {
                    "suggestion": "content",
                    "score": 1
                }
            ]
        }
    ],
    "correctionType": "High"
}


In [112]:
all_data = read_data('data/CAp/trainingSet_3000_v3_fixed.txt')

https://t.co/xkkIWUymCOO

Reading is done with 1 errors


In [113]:
test = read_data('data/CAp/test_set_wlabels.txt')

In [22]:
len(all_data[82][0]), len(all_data[82][1])

(12, 12)

In [23]:
untokenize(all_data[82][0])

"dantesque, une visite au cimetière de laeken s'impose @museerodinparis"

In [24]:
output

{'_type': 'SpellCheck',
 'correctionType': 'High',
 'flaggedTokens': [{'offset': 55,
   'suggestions': [{'score': 1, 'suggestion': 'musee rodin paris'}],
   'token': 'museerodinparis',
   'type': 'UnknownToken'}]}

In [20]:
all_data[82]

(['dantesque',
  ',',
  'une',
  'visite',
  'au',
  'cimetière',
  'de',
  'laeken',
  's',
  "'",
  'impose',
  '@museerodinparis'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geoloc', 'O', 'O', 'O', 'O'])

In [59]:
len(fixed)

2624

In [57]:
spellsafe(' '.join(tokens), output)

token:  rera 	suggest:  rer a
token:  qml #uxol75 	suggest:  qmluxol75


"j' ai un #rera \\ o / go cergy quand il veut #qml #uxol75"

In [ ]:
fixed = []

In [63]:
for tokens, tags in tqdm(all_data[2624:]):
    data = {'text': untokenize(tokens)}
    body = urllib.parse.urlencode(data)
    conn.request ("POST", path + params, body, headers)
    response = conn.getresponse()
    output = json.loads(response.read())
    
    fixed_text = spellsafe(' '.join(tokens), output)
    fixed_tokens = fixed_text.split(' ')
    assert len(fixed_tokens) == len(tags), output

    fixed.append((fixed_tokens, tags))
    sleep(1/100.)

  1%|          | 2/377 [00:00<00:57,  6.57it/s]

token:  rera 	suggest:  rer a
token:  qml #uxol75 	suggest:  qmluxol75
token:  rerb 	suggest:  rer b


  1%|          | 4/377 [00:00<00:55,  6.67it/s]

token:  #rer 	suggest:  rer b
token:   stationn 	suggest:  stationner
token:   cite 	suggest:  cite u


  2%|▏         | 9/377 [00:01<00:53,  6.88it/s]

...

 98%|█████████▊| 371/377 [00:58<00:00,  6.32it/s]

token:  rerb 	suggest:  rer b
token:  rerd 	suggest:  rer d


100%|█████████▉| 376/377 [00:59<00:00,  6.34it/s]

token:  cagade 	suggest:  cage de
token:  lignej 	suggest:  ligne j


100%|██████████| 377/377 [00:59<00:00,  6.34it/s]


In [65]:
with open('../data/CAp/fixed.pkl', 'wb') as f:
    pickle.dump(fixed, f)

In [64]:
len(fixed)

3001

In [66]:
len(fixed) == len(all_data)

True

In [67]:
all_data = read_data('../data/CAp/test_set_wlabels.txt')

In [70]:
fixed = []

for tokens, tags in tqdm(all_data[2624:]):
    data = {'text': untokenize(tokens)}
    body = urllib.parse.urlencode(data)
    conn.request ("POST", path + params, body, headers)
    response = conn.getresponse()
    output = json.loads(response.read())
    
    fixed_text = spellsafe(' '.join(tokens), output)
    fixed_tokens = fixed_text.split(' ')
    assert len(fixed_tokens) == len(tags), output

    fixed.append((fixed_tokens, tags))
    sleep(1/100.)

  5%|▍         | 52/1066 [00:07<02:20,  7.19it/s]

token:   marvy 	suggest:  marvin


  5%|▌         | 55/1066 [00:07<02:19,  7.24it/s]

token:  rt @darlingloraine: #mardiconseil #surlaroute le rouleau original #jackkerouac @folio_livres https://t.co/am4kuz0f7 	suggest:  rt darling lorraine mardi conseil sur la route le rouleau original jack kerouac folio livres https t am4kuz0f7q


  6%|▌         | 66/1066 [00:09<02:18,  7.22it/s]

...

100%|█████████▉| 1061/1066 [02:34<00:00,  6.88it/s]

token:  ladefense 	suggest:  la defense
token:  rera 	suggest:  rer a
token:  rera 	suggest:  rer a


100%|█████████▉| 1063/1066 [02:34<00:00,  6.88it/s]

token:  rera 	suggest:  rer a


100%|██████████| 1066/1066 [02:34<00:00,  6.88it/s]


In [71]:
with open('../data/CAp/test_spellchecked.pkl', 'wb') as f:
    pickle.dump(fixed, f)

# Loading and splitting train set

In [96]:
with open('data/CAp/fixed.pkl', 'rb') as f:
    fixed = pickle.load(f)

In [114]:
# for original dataset:
fixed = all_data

In [121]:
VAL_SIZE = 0.15

indices = list(range(len(fixed)))
np.random.shuffle(indices)

val_idx = indices[:int(len(indices) * VAL_SIZE)]
train_idx = indices[len(val_idx):]

train = []
valid = []

for i in train_idx:
    train.append(fixed[i])

for i in val_idx:
    valid.append(fixed[i])

In [122]:
len(train), len(valid)

(2551, 450)

In [109]:
train[0]

(['@theedadr',
  '@gaton__',
  'la',
  'c’est',
  'bon',
  'on',
  'devrait',
  'pas',
  'le',
  'voir',
  'avant',
  '#',
  'ans',
  'minimum',
  '😂'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])

In [110]:
valid[0]

(['purger',
  'sa',
  'peine',
  'au',
  'plus',
  'proche',
  'des',
  'siens',
  '.',
  'https://t.co/nrwm5p241u'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])

In [83]:
with open('../data/CAp/test_spellchecked.pkl', 'rb') as f:
    test = pickle.load(f)

In [84]:
len(test)

1066

In [125]:
test[1]

(['rt',
  '@cestrosi',
  ':',
  'ce',
  'midi',
  ',',
  'j',
  "'",
  'ai',
  'rappelé',
  'notre',
  'volonté',
  'sans',
  'faille',
  'de',
  'construire',
  'ligne',
  'à',
  'grande',
  'vitesse',
  'ds',
  'notre',
  'région',
  '#projetlnpca',
  'https://t',
  '.',
  'c',
  '…'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'])

In [123]:
dataset_dict = {'train': train, 'valid': valid, 'test': test}

In [126]:
with open('data/CAp/dataset_original_dict.pkl', 'wb') as f:
    pickle.dump(dataset_dict, f)

In [87]:
with open('../data/CAp/dataset_dict.pkl', 'wb') as f:
    pickle.dump(dataset_dict, f)